In [1]:
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
import math
import numpy as np

In [2]:
class MoneyModel(Model):
    """A model with some number of agents."""
    def __init__(self, N, width, height):
        self.num_agents = N
        self.grid = MultiGrid(width, height, True)
        self.schedule = RandomActivation(self)

        # Create agents
        for i in range(self.num_agents):
            a = MoneyAgent(i, self)
            self.schedule.add(a)
            # Add the agent to a random grid cell
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            self.grid.place_agent(a, (x, y))
            infected = np.random.choice([0,1], p=[0.90,0.10])
            if infected == 1:
                a.state = 1

        self.datacollector = DataCollector(
            agent_reporters={"State": "state"})

    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()

In [3]:
def compute_gini(model):
    agent_wealths = [agent.affected for agent in model.schedule.agents]
    x = sorted(agent_wealths)
    N = model.num_agents
    B = sum( xi * (N-i) for i,xi in enumerate(x) ) / (N*sum(x))
    return (1 + (1/N) - 2*B)

In [4]:
def create_day(model):
    h = model.grid.height
    w = model.grid.width

    print("Parks")
    for row in range(h//2):
        for col in range(w):
            state = model.grid.get_cell_list_contents((row,col))
            if state:
                print("*", end = " ")
            else:
                print("-", end = " ")
        print()
    
    print("Grocery shops")
    for row in range(h//2, h):
        for col in range(w):
            state = model.grid.get_cell_list_contents((row,col))
            if state:
                print("*", end = " ")
            else:
                print("-", end = " ")
        print()

In [5]:
# h = model.grid.height
# w = model.grid.width
# [(row, col) for row in range(h//2) for col in range(w)]

In [6]:
def get_places(model):
    parks = [(row, col) for row in range(h//2) for col in range(w)]
    grocery = [(row, col) for row in range(h//2, h) for col in range(w)]
    return [parks, grocery]

In [7]:
def get_infections(model):
    inf = 0
    for agent in model.schedule.agents:
        if agent.state == 1:
            inf += 1
    return inf

In [8]:
class MoneyAgent(Agent):
    """ An agent with fixed initial wealth."""
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.state = 0

    def move(self):
        possible_steps = self.model.grid.get_neighborhood(
            self.pos,
            moore=True,
            include_center=False)
        new_position = self.random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)

    def infect(self):
        cellmates = self.model.grid.get_cell_list_contents([self.pos])
        if len(cellmates) > 1:
            other = self.random.choice(cellmates)
            if self.state == 1:
                other.state = 1
    
    def home(self):
        if self.state == 1:
            family = [elem for elem in range(math.floor(self.unique_id)*4, math.floor(self.unique_id)*4+4)]
            for member in family:
                if member < model.num_agents:
                    self.model.schedule.agents[member].state = 1
    
    def step(self):
        self.move()
        self.infect()
        self.home()

In [21]:
model = MoneyModel(1000, 100, 60)

In [22]:
days = 0
while(get_infections(model) != model.num_agents):
    model.step()
#     create_day(model)
#     get_infections(model)
    days += 1
print("Days to death", days)

Days to death 30


In [11]:
# x = model.grid.get_cell_list_contents((0,7))

In [12]:
# for cell in model.grid.coord_iter():
#     print(cell)

In [18]:
for i in model.schedule.agents:
    print(i.state)
#     print(i.unique_id)
# len(model.schedule.agents)

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [14]:
# import numpy as np
# import matplotlib.pyplot as plt

# agent_counts = np.zeros((model.grid.width, model.grid.height))
# for cell in model.grid.coord_iter():
#     cell_content, x, y = cell
#     agent_count = len(cell_content)
#     agent_counts[x][y] = agent_count
# plt.imshow(agent_counts, interpolation='nearest')
# plt.colorbar()

In [15]:
# gini = model.datacollector.get_model_vars_dataframe()
# gini.plot()

In [16]:
# agent_wealth = model.datacollector.get_agent_vars_dataframe()
# agent_wealth.head()

In [17]:
# end_wealth = agent_wealth.xs(19, level="Step")["Wealth"]
# end_wealth.hist(bins=range(agent_wealth.Wealth.max()+1))